##### Audio Classification Using Edge Impulse
This example uses audio input from fast analog input IN2 and uses local Edge Impulse Classification to classigy 5 seconds of audio.

This is based on the Edge Immpulse example:
https://github.com/edgeimpulse/linux-sdk-python/blob/master/examples/audio/classify.py
The following are required to use the edge_impulse_linux:  
<code>
apt install libatlas-base-dev libportaudio0 libportaudio2 libportaudiocpp0 portaudio19-dev python3-opencv
pip3 install PyAudio
pip3 install edge_impulse_linux</code>

In [1]:
import os
import sys, getopt
import signal
import numpy as np
from edge_impulse_linux.runner import ImpulseRunner

model_file_name = 'modelfile.eim'
runner = None
dir_path = os.path.dirname(os.path.realpath(model_file_name))
modelfile = os.path.join(dir_path, model_file_name)
modelfile

'/root/red-pitaya-projects/jupyter-notebooks/edge_impulse_audio/modelfile.eim'

In [2]:
# Add the files you want to upload to Edge Impulse
files = ['float_wave_test_raw.wav']
audio_file = files[0] #save the first one

# Creating the data payload for the request.
files=(('data', (os.path.basename(i), open(i, 'rb'), 'audio/wav')) for i in files)
audio_file

'float_wave_test_raw.wav'

In [3]:
runner = None

# Add the files you want to upload to Edge Impulse
files = ['float_wave_test_raw.wav']
audio_file = files[0] #save the first one

# Creating the data payload for the request.
files=(('data', (os.path.basename(i), open(i, 'rb'), 'audio/wav')) for i in files)
audio_file

model_file_name = 'modelfile.eim'

dir_path = os.path.dirname(os.path.realpath(model_file_name))
modelfile = os.path.join(dir_path, model_file_name)
full_audio_file = os.path.join(dir_path, audio_file)

full_audio_file
files=(('data', (os.path.basename(i), open(i, 'rb'), 'audio/wav')) for i in files)
files

<generator object <genexpr> at 0xb549d060>

In [4]:
from scipy.io import wavfile
samplerate, generator = wavfile.read(full_audio_file)

features = np.array([], dtype=np.int16)
runner = ImpulseRunner(modelfile)

model_info = runner.init()
labels = model_info['model_parameters']['labels']
window_size = model_info['model_parameters']['input_features_count']
sampling_rate = model_info['model_parameters']['frequency']
print('Loaded runner for "' + model_info['project']['owner'] + ' / ' + model_info['project']['name'] + '"')
print(labels)
print(window_size)
print(sampling_rate)

Loaded runner for "John Bracegirdle / Sound Test"
['background-noise', 'edge-impulse', 'red-pitaya']
15000
3000


In [5]:
samplerate, generator = wavfile.read(full_audio_file)
features = np.concatenate((features, generator), axis=0)
res = runner.classify(features[:window_size].tolist())
res

{'result': {'classification': {'background-noise': 0.9999896287918091,
   'edge-impulse': 6.324415153358132e-06,
   'red-pitaya': 4.0374552554567344e-06}},
 'timing': {'anomaly': 0,
  'classification': 4,
  'dsp': 24,
  'json': 16,
  'stdin': 31}}